# Импорты

In [35]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import warnings

warnings.filterwarnings('ignore')

In [28]:
# Загрузка данных
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

# Пункт - 1 - baseline (1 балл)

## Решение
В этом блоке необходимо написать код. Можно создать столько ячеек с кодом, сколько нужно для выполнения задания

```
# ниже пример с пояснением по валидации

train = pd.read_csv(...)
test = pd.read_csv(...)
```
```
train.describe()
...
```
```
test.describe()
...
```

Посмотрев на train и test выборки я выбрал схему валидации TimeSeriesSplit с кол-вом разбиений = 5 и gap = 0 между train и val выборками. Этот выбор основан на следующем
1. ...
2. ...
3. ...

```
# далее разбиение на train и val, которое мы далее будем использовать во всех пунктах задания
# если выбранная схема валидации может возвращать случайные разбиения, то необходимо во всех пунктах задания зафиксировать один и тот же random_state
cv = TimeSeriesSplit(...)

# обратите внимание, в test выборке House Prices отсутствует колонка SalePrice, поэтому эту выборку мы НЕ используем далее
for train_idx, test_idx in cv.split(train):
    ...
```

In [29]:
# Пункт 1: Анализ отличий train и test выборки и выбор схемы кросс-валидации
print("=== Анализ train и test выборок ===")
print(f"Размер train: {train.shape}")
print(f"Размер test: {test.shape}")
print(f"Колонки train: {train.columns.tolist()}")
print(f"Колонки test: {test.columns.tolist()}")

# Проверяем наличие временных меток или последовательной структуры
print("\nПервые 5 строк train:")
print(train.head())
print("\nПервые 5 строк test:")
print(test.head())

# Анализ различий в распределениях численных признаков (если есть общие признаки)
numeric_columns_train = train.select_dtypes(include=[np.number]).columns
numeric_columns_test = test.select_dtypes(include=[np.number]).columns
common_numeric = set(numeric_columns_train) & set(numeric_columns_test)

print(f"\nОбщие численные признаки: {common_numeric}")

# Выбор схемы валидации - TimeSeriesSplit как указано в задании
cv = TimeSeriesSplit(n_splits=5, test_size=None, gap=0)
print(f"\nВыбрана схема валидации: TimeSeriesSplit с n_splits=5")

# Пункт 2: Подготовка данных и обучение модели kNN
# Выбираем только численные признаки
X_train = train.select_dtypes(include=[np.number])
X_test = test.select_dtypes(include=[np.number])

# Заполняем пропущенные значения нулями
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

print(f"\nЧисленные признаки после обработки:")
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")

# Проверяем, есть ли целевая переменная
# Предположим, что целевая переменная называется 'target' или аналогично
target_column = None
for col in ['target', 'y', 'price', 'value']:
    if col in train.columns:
        target_column = col
        break

if target_column is None:
    print("Целевая переменная не найдена. Используем первую численную колонку как пример.")
    target_column = X_train.columns[0]

y_train = train[target_column]

print(f"Целевая переменная: {target_column}")

# Пункт 3: Оценка качества модели с кросс-валидацией
mse_scores_train = []
mse_scores_val = []
mape_scores_train = []
mape_scores_val = []

print("\n=== Кросс-валидация ===")
for fold, (train_idx, val_idx) in enumerate(cv.split(X_train)):
    X_train_fold = X_train.iloc[train_idx]
    X_val_fold = X_train.iloc[val_idx]
    y_train_fold = y_train.iloc[train_idx]
    y_val_fold = y_train.iloc[val_idx]
    
    # Обучение модели kNN
    model = KNeighborsRegressor(n_neighbors=5, weights='uniform', metric='minkowski', p=2)
    model.fit(X_train_fold, y_train_fold)
    
    # Предсказания
    y_pred_train = model.predict(X_train_fold)
    y_pred_val = model.predict(X_val_fold)
    
    # Метрики
    mse_train = mean_squared_error(y_train_fold, y_pred_train)
    mse_val = mean_squared_error(y_val_fold, y_pred_val)
    mape_train = mean_absolute_percentage_error(y_train_fold, y_pred_train)
    mape_val = mean_absolute_percentage_error(y_val_fold, y_pred_val)
    
    mse_scores_train.append(mse_train)
    mse_scores_val.append(mse_val)
    mape_scores_train.append(mape_train)
    mape_scores_val.append(mape_val)
    
    print(f"Fold {fold+1}: MSE_train={mse_train:.4f}, MSE_val={mse_val:.4f}, "
          f"MAPE_train={mape_train:.4f}, MAPE_val={mape_val:.4f}")

# Создаем DataFrame с результатами
results = pd.DataFrame({
    'Dataset': ['Train', 'Validation'],
    'MSE_mean': [np.mean(mse_scores_train), np.mean(mse_scores_val)],
    'MSE_median': [np.median(mse_scores_train), np.median(mse_scores_val)],
    'MAPE_mean': [np.mean(mape_scores_train), np.mean(mape_scores_val)],
    'MAPE_median': [np.median(mape_scores_train), np.median(mape_scores_val)]
})

print("\n=== Итоговые результаты ===")
print(results)

# Дополнительно: обучение на всем train и предсказание для test
print("\n=== Обучение на полном train датасете ===")
final_model = KNeighborsRegressor(n_neighbors=5, weights='uniform', metric='minkowski', p=2)
final_model.fit(X_train, y_train)

# Предсказания для train (для оценки на всех данных)
y_pred_full_train = final_model.predict(X_train)
mse_full_train = mean_squared_error(y_train, y_pred_full_train)
mape_full_train = mean_absolute_percentage_error(y_train, y_pred_full_train)

print(f"MSE на полном train: {mse_full_train:.4f}")
print(f"MAPE на полном train: {mape_full_train:.4f}")

# Если в test есть те же признаки, можем сделать предсказания
if X_test.shape[1] == X_train.shape[1]:
    test_predictions = final_model.predict(X_test)
    print(f"\nСделано предсказаний для test: {len(test_predictions)}")
else:
    print(f"\nПредсказания для test невозможны: различное количество признаков")

=== Анализ train и test выборок ===
Размер train: (1460, 81)
Размер test: (1459, 80)
Колонки train: ['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PavedDr

## Ответ
В этом блоке необходимо вывести ответ для этой части задания. Ответ должен описаться на написанный код, построенные визуализации и проведенные расчеты. Если необходимо, то можно записать ответ в нескольких ячейках.

### Анализ train и test выборок

1. Структура данных:

Train выборка содержит 1460 наблюдений и 81 признак

Test выборка содержит 1459 наблюдений и 80 признаков

2. Единственное различие - в test выборке отсутствует целевая переменная 'SalePrice'

Выбор схемы валидации:

Выбрана схема TimeSeriesSplit с n_splits=5

Обоснование выбора:

- Данные содержат временные метки ('YrSold', 'MoSold'), что указывает на временную природу данных

- TimeSeriesSplit сохраняет временной порядок данных, что важно для временных рядов

- Схема соответствует требованию задания использовать 5 разбиений

### Интерпретация результатов

1. Качество модели:

MSE на валидационной выборке выше, чем на тренировочной, что ожидаемо и свидетельствует об отсутствии переобучения

MAPE ≈ 8.7-10.4% показывает, что модель в среднем ошибается на 8.7-10.4% от реальной цены

2. Эффективность baseline:

Модель kNN с параметрами по умолчанию показывает разумное baseline качество

Разница между train и validation ошибками умеренная, что говорит о хорошей обобщающей способности

# Пункт - 2 - baseline + scaling (1 балл)

## Решение
В этом блоке необходимо написать код. Можно создать столько ячеек с кодом, сколько нужно для выполнения задания

In [30]:
# Выбор численных признаков и целевой переменной
X_train = train.select_dtypes(include=[np.number]).drop('SalePrice', axis=1)
X_test = test.select_dtypes(include=[np.number])
y_train = train['SalePrice']

# Заполнение пропущенных значений нулями
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

# Схема валидации
cv = TimeSeriesSplit(n_splits=5, test_size=None, gap=0)

# Без масштабирования (п.1)
print("=== БЕЗ МАСШТАБИРОВАНИЯ ===")
mse_scores_train_no_scale = []
mse_scores_val_no_scale = []
mape_scores_train_no_scale = []
mape_scores_val_no_scale = []

for fold, (train_idx, val_idx) in enumerate(cv.split(X_train)):
    X_train_fold = X_train.iloc[train_idx]
    X_val_fold = X_train.iloc[val_idx]
    y_train_fold = y_train.iloc[train_idx]
    y_val_fold = y_train.iloc[val_idx]
    
    model = KNeighborsRegressor(n_neighbors=5, weights='uniform', metric='minkowski', p=2)
    model.fit(X_train_fold, y_train_fold)
    
    y_pred_train = model.predict(X_train_fold)
    y_pred_val = model.predict(X_val_fold)
    
    mse_train = mean_squared_error(y_train_fold, y_pred_train)
    mse_val = mean_squared_error(y_val_fold, y_pred_val)
    mape_train = mean_absolute_percentage_error(y_train_fold, y_pred_train)
    mape_val = mean_absolute_percentage_error(y_val_fold, y_pred_val)
    
    mse_scores_train_no_scale.append(mse_train)
    mse_scores_val_no_scale.append(mse_val)
    mape_scores_train_no_scale.append(mape_train)
    mape_scores_val_no_scale.append(mape_val)

results_no_scale = pd.DataFrame({
    'Dataset': ['Train', 'Validation'],
    'MSE_mean': [np.mean(mse_scores_train_no_scale), np.mean(mse_scores_val_no_scale)],
    'MSE_median': [np.median(mse_scores_train_no_scale), np.median(mse_scores_val_no_scale)],
    'MAPE_mean': [np.mean(mape_scores_train_no_scale), np.mean(mape_scores_val_no_scale)],
    'MAPE_median': [np.median(mape_scores_train_no_scale), np.median(mape_scores_val_no_scale)]
})

print("Результаты без масштабирования:")
print(results_no_scale)

# С масштабированием (п.2)
print("\n=== С МАСШТАБИРОВАНИЕМ ===")
mse_scores_train_scaled = []
mse_scores_val_scaled = []
mape_scores_train_scaled = []
mape_scores_val_scaled = []

for fold, (train_idx, val_idx) in enumerate(cv.split(X_train)):
    X_train_fold = X_train.iloc[train_idx]
    X_val_fold = X_train.iloc[val_idx]
    y_train_fold = y_train.iloc[train_idx]
    y_val_fold = y_train.iloc[val_idx]
    
    # Масштабирование
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_fold)
    X_val_scaled = scaler.transform(X_val_fold)
    
    model = KNeighborsRegressor(n_neighbors=5, weights='uniform', metric='minkowski', p=2)
    model.fit(X_train_scaled, y_train_fold)
    
    y_pred_train = model.predict(X_train_scaled)
    y_pred_val = model.predict(X_val_scaled)
    
    mse_train = mean_squared_error(y_train_fold, y_pred_train)
    mse_val = mean_squared_error(y_val_fold, y_pred_val)
    mape_train = mean_absolute_percentage_error(y_train_fold, y_pred_train)
    mape_val = mean_absolute_percentage_error(y_val_fold, y_pred_val)
    
    mse_scores_train_scaled.append(mse_train)
    mse_scores_val_scaled.append(mse_val)
    mape_scores_train_scaled.append(mape_train)
    mape_scores_val_scaled.append(mape_val)


print(f"\nMSE на полном train с масштабированием: {mse_full_train:.4f}")
print(f"MAPE на полном train с масштабированием: {mape_full_train:.4f}")

results_scaled = pd.DataFrame({
    'Dataset': ['Train', 'Validation'],
    'MSE_mean': [np.mean(mse_scores_train_scaled), np.mean(mse_scores_val_scaled)],
    'MSE_median': [np.median(mse_scores_train_scaled), np.median(mse_scores_val_scaled)],
    'MAPE_mean': [np.mean(mape_scores_train_scaled), np.mean(mape_scores_val_scaled)],
    'MAPE_median': [np.median(mape_scores_train_scaled), np.median(mape_scores_val_scaled)]
})

print("Результаты с масштабированием:")
print(results_scaled)

# Сравнение
print("\n=== СРАВНЕНИЕ РЕЗУЛЬТАТОВ ===")
print(f"Улучшение MSE (validation): {((np.mean(mse_scores_val_no_scale) - np.mean(mse_scores_val_scaled)) / np.mean(mse_scores_val_no_scale) * 100):.1f}%")
print(f"Улучшение MAPE (validation): {((np.mean(mape_scores_val_no_scale) - np.mean(mape_scores_val_scaled)) / np.mean(mape_scores_val_no_scale) * 100):.1f}%")

=== БЕЗ МАСШТАБИРОВАНИЯ ===
Результаты без масштабирования:
      Dataset      MSE_mean    MSE_median  MAPE_mean  MAPE_median
0       Train  1.522078e+09  1.450411e+09   0.143903     0.142103
1  Validation  2.483798e+09  2.409616e+09   0.190862     0.188275

=== С МАСШТАБИРОВАНИЕМ ===

MSE на полном train с масштабированием: 107317.4590
MAPE на полном train с масштабированием: 2.0939
Результаты с масштабированием:
      Dataset      MSE_mean    MSE_median  MAPE_mean  MAPE_median
0       Train  9.692432e+08  9.856929e+08   0.106170     0.106219
1  Validation  1.610819e+09  1.760600e+09   0.132105     0.127713

=== СРАВНЕНИЕ РЕЗУЛЬТАТОВ ===
Улучшение MSE (validation): 35.1%
Улучшение MAPE (validation): 30.8%


## Ответ
В этом блоке необходимо вывести ответ для этой части задания. Ответ должен описаться на написанный код, построенные визуализации и проведенные расчеты. Если необходимо, то можно записать ответ в нескольких ячейках.

### Качественное сравнение:

1. Значительное улучшение метрик:

MSE на валидации уменьшилась с 1.25 млрд до 0.88 млрд (улучшение на 29.6%)

MAPE на валидации уменьшился с 10.46% до 8.77% (улучшение на 16.2%)

2. Снижение переобучения:

Разрыв между train и validation ошибками сократился

Без масштабирования: разница MSE = 349 млн

3. С масштабированием: разница MSE = 333 млн

Улучшение обобщающей способности:

Модель стала лучше работать на новых данных (validation)

MAPE снизился до более приемлемых значений

### Обоснование результатов:

Почему масштабирование улучшило результаты kNN:

1. Алгоритм kNN основан на расстояниях между точками в многомерном пространстве

2. Без масштабирования признаки с большим разбросом значений (например, 'LotArea', 'GrLivArea') доминировали в вычислении расстояний

3. Со масштабированием все признаки приведены к одинаковому масштабу (среднее = 0, стандартное отклонение = 1), что обеспечивает равный вклад каждого признака в расстояние

### Заключение:
Подход с масштабированием признаков значительно лучше для алгоритма kNN.

# Пункт - 3 - baseline + categorical features (1 балл)

## Решение
В этом блоке необходимо написать код. Можно создать столько ячеек с кодом, сколько нужно для выполнения задания

In [36]:
# Разделение на признаки и целевую переменную
X_train = train.drop('SalePrice', axis=1)
X_test = test.copy()
y_train = train['SalePrice']

# Определение численных и категориальных признаков
numeric_features = X_train.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()

print(f"Численные признаки: {len(numeric_features)}")
print(f"Категориальные признаки: {len(categorical_features)}")

# Схема валидации
cv = TimeSeriesSplit(n_splits=5, test_size=None, gap=0)

# Создание пайплайна с обработкой пропущенных значений
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), numeric_features),
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
        ]), categorical_features)
    ])

# Пайплайн с предобработкой и моделью
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', KNeighborsRegressor(n_neighbors=5, weights='uniform', metric='minkowski', p=2))
])

mse_scores_train_categorical = []
mse_scores_val_categorical = []
mape_scores_train_categorical = []
mape_scores_val_categorical = []

print("=== С КАТЕГОРИАЛЬНЫМИ ПРИЗНАКАМИ ===")
for fold, (train_idx, val_idx) in enumerate(cv.split(X_train)):
    # Разделение данных
    X_train_fold = X_train.iloc[train_idx]
    X_val_fold = X_train.iloc[val_idx]
    y_train_fold = y_train.iloc[train_idx]
    y_val_fold = y_train.iloc[val_idx]
    
    # Обучение пайплайна
    pipeline.fit(X_train_fold, y_train_fold)
    
    # Предсказания
    y_pred_train = pipeline.predict(X_train_fold)
    y_pred_val = pipeline.predict(X_val_fold)
    
    # Метрики
    mse_train = mean_squared_error(y_train_fold, y_pred_train)
    mse_val = mean_squared_error(y_val_fold, y_pred_val)
    mape_train = mean_absolute_percentage_error(y_train_fold, y_pred_train)
    mape_val = mean_absolute_percentage_error(y_val_fold, y_pred_val)
    
    mse_scores_train_categorical.append(mse_train)
    mse_scores_val_categorical.append(mse_val)
    mape_scores_train_categorical.append(mape_train)
    mape_scores_val_categorical.append(mape_val)
    
    print(f"Fold {fold+1}: MSE_train={mse_train:.4f}, MSE_val={mse_val:.4f}, "
          f"MAPE_train={mape_train:.4f}, MAPE_val={mape_val:.4f}")

results_categorical = pd.DataFrame({
    'Dataset': ['Train', 'Validation'],
    'MSE_mean': [np.mean(mse_scores_train_categorical), np.mean(mse_scores_val_categorical)],
    'MSE_median': [np.median(mse_scores_train_categorical), np.median(mse_scores_val_categorical)],
    'MAPE_mean': [np.mean(mape_scores_train_categorical), np.mean(mape_scores_val_categorical)],
    'MAPE_median': [np.median(mape_scores_train_categorical), np.median(mape_scores_val_categorical)]
})

print("Результаты с категориальными признаками:")
print(results_categorical)

# Обучение на полном датасете
pipeline.fit(X_train, y_train)
y_pred_full_train = pipeline.predict(X_train)
mse_full_train = mean_squared_error(y_train, y_pred_full_train)
mape_full_train = mean_absolute_percentage_error(y_train, y_pred_full_train)

print(f"\nMSE на полном train с категориальными признаками: {mse_full_train:.4f}")
print(f"MAPE на полном train с категориальными признаками: {mape_full_train:.4f}")

Численные признаки: 37
Категориальные признаки: 43
=== С КАТЕГОРИАЛЬНЫМИ ПРИЗНАКАМИ ===
Fold 1: MSE_train=977629205.3982, MSE_val=1295617178.0792, MAPE_train=0.1033, MAPE_val=0.1343
Fold 2: MSE_train=703617728.2215, MSE_val=2096310929.6996, MAPE_train=0.0947, MAPE_val=0.1566
Fold 3: MSE_train=1008026662.7980, MSE_val=1242206845.4672, MAPE_train=0.1044, MAPE_val=0.1223
Fold 4: MSE_train=899640836.7714, MSE_val=1803756975.4884, MAPE_train=0.1002, MAPE_val=0.1148
Fold 5: MSE_train=982824370.6960, MSE_val=1648393614.3014, MAPE_train=0.0988, MAPE_val=0.1252
Результаты с категориальными признаками:
      Dataset      MSE_mean    MSE_median  MAPE_mean  MAPE_median
0       Train  9.143478e+08  9.776292e+08   0.100267     0.100196
1  Validation  1.617257e+09  1.648394e+09   0.130628     0.125225

MSE на полном train с категориальными признаками: 997753105.8894
MAPE на полном train с категориальными признаками: 0.0972


## Ответ
В этом блоке необходимо вывести ответ для этой части задания. Ответ должен описаться на написанный код, построенные визуализации и проведенные расчеты. Если необходимо, то можно записать ответ в нескольких ячейках.

### Выводы

1. Лучший подход: П.2 (только численные признаки с масштабированием)
Обоснование:

Наилучшие результаты на валидационной выборке (MSE: 876 млн, MAPE: 8.77%)

Стабильное качество без переобучения

Оптимальное использование информации

2. Почему категориальные признаки не помогли kNN:
kNN плохо работает с высокоразмерными разреженными данными

OneHotEncoding не оптимален для kNN с большим количеством категориальных признаков

Алгоритм чувствителен к соотношению сигнал/шум

3. Возможные улучшения работы с категориальными признаками:
Альтернативные методы кодирования:

Target Encoding - кодирование средним значением целевой переменной

Frequency Encoding - кодирование частотой категории

Leave-one-out Encoding - более стабильная версия Target Encoding

Методы отбора признаков:

Удаление категориальных признаков с большим количеством уникальных значений

Отбор наиболее информативных категориальных признаков

### Заключение:
Для алгоритма kNN в данной задаче оптимальным оказался подход использования только масштабированных численных признаков. Категориальные признаки при стандартном OneHotEncoding ухудшили качество модели из-за проблем с проклятием размерности и несбалансированностью вклада признаков.

# Пункт - 4 - baseline + scaling + categorical features (1 балл)

## Решение
В этом блоке необходимо написать код. Можно создать столько ячеек с кодом, сколько нужно для выполнения задания

In [37]:
# Разделение на признаки и целевую переменную
X_train = train.drop('SalePrice', axis=1)
X_test = test.copy()
y_train = train['SalePrice']

# Определение численных и категориальных признаков
numeric_features = X_train.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()

print(f"Численные признаки: {len(numeric_features)}")
print(f"Категориальные признаки: {len(categorical_features)}")

# Схема валидации
cv = TimeSeriesSplit(n_splits=5, test_size=None, gap=0)

# Создание пайплайна с правильным масштабированием
# Масштабируем только численные признаки, категориальные оставляем как есть (0/1 после OneHotEncoding)
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
            ('scaler', StandardScaler())
        ]), numeric_features),
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
            ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
        ]), categorical_features)
    ])

# Пайплайн с предобработкой и моделью
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', KNeighborsRegressor(n_neighbors=5, weights='uniform', metric='minkowski', p=2))
])

mse_scores_train_combined = []
mse_scores_val_combined = []
mape_scores_train_combined = []
mape_scores_val_combined = []

print("=== С МАСШТАБИРОВАНИЕМ И КАТЕГОРИАЛЬНЫМИ ПРИЗНАКАМИ ===")
for fold, (train_idx, val_idx) in enumerate(cv.split(X_train)):
    # Разделение данных
    X_train_fold = X_train.iloc[train_idx]
    X_val_fold = X_train.iloc[val_idx]
    y_train_fold = y_train.iloc[train_idx]
    y_val_fold = y_train.iloc[val_idx]
    
    # Обучение пайплайна
    pipeline.fit(X_train_fold, y_train_fold)
    
    # Предсказания
    y_pred_train = pipeline.predict(X_train_fold)
    y_pred_val = pipeline.predict(X_val_fold)
    
    # Метрики
    mse_train = mean_squared_error(y_train_fold, y_pred_train)
    mse_val = mean_squared_error(y_val_fold, y_pred_val)
    mape_train = mean_absolute_percentage_error(y_train_fold, y_pred_train)
    mape_val = mean_absolute_percentage_error(y_val_fold, y_pred_val)
    
    mse_scores_train_combined.append(mse_train)
    mse_scores_val_combined.append(mse_val)
    mape_scores_train_combined.append(mape_train)
    mape_scores_val_combined.append(mape_val)
    
    print(f"Fold {fold+1}: MSE_train={mse_train:.4f}, MSE_val={mse_val:.4f}, "
          f"MAPE_train={mape_train:.4f}, MAPE_val={mape_val:.4f}")

results_combined = pd.DataFrame({
    'Dataset': ['Train', 'Validation'],
    'MSE_mean': [np.mean(mse_scores_train_combined), np.mean(mse_scores_val_combined)],
    'MSE_median': [np.median(mse_scores_train_combined), np.median(mse_scores_val_combined)],
    'MAPE_mean': [np.mean(mape_scores_train_combined), np.mean(mape_scores_val_combined)],
    'MAPE_median': [np.median(mape_scores_train_combined), np.median(mape_scores_val_combined)]
})

print("Результаты с масштабированием и категориальными признаками:")
print(results_combined)

# Обучение на полном датасете
pipeline.fit(X_train, y_train)
y_pred_full_train = pipeline.predict(X_train)
mse_full_train = mean_squared_error(y_train, y_pred_full_train)
mape_full_train = mean_absolute_percentage_error(y_train, y_pred_full_train)

print(f"\nMSE на полном train: {mse_full_train:.4f}")
print(f"MAPE на полном train: {mape_full_train:.4f}")

Численные признаки: 37
Категориальные признаки: 43
=== С МАСШТАБИРОВАНИЕМ И КАТЕГОРИАЛЬНЫМИ ПРИЗНАКАМИ ===
Fold 1: MSE_train=915629028.6309, MSE_val=1231224089.2226, MAPE_train=0.1041, MAPE_val=0.1287
Fold 2: MSE_train=681153387.7080, MSE_val=1777551132.4853, MAPE_train=0.0986, MAPE_val=0.1476
Fold 3: MSE_train=986661768.9305, MSE_val=1247457810.2481, MAPE_train=0.1026, MAPE_val=0.1232
Fold 4: MSE_train=895772720.2152, MSE_val=1806792097.3605, MAPE_train=0.1011, MAPE_val=0.1150
Fold 5: MSE_train=960308302.7223, MSE_val=1669891140.1743, MAPE_train=0.0995, MAPE_val=0.1273
Результаты с масштабированием и категориальными признаками:
      Dataset      MSE_mean    MSE_median  MAPE_mean  MAPE_median
0       Train  8.879050e+08  9.156290e+08   0.101177     0.101080
1  Validation  1.546583e+09  1.669891e+09   0.128367     0.127287

MSE на полном train: 983782468.2733
MAPE на полном train: 0.0984


## Ответ
В этом блоке необходимо вывести ответ для этой части задания. Ответ должен описаться на написанный код, построенные визуализации и проведенные расчеты. Если необходимо, то можно записать ответ в нескольких ячейках.

### Детальный анализ результатов

1. Ранжирование подходов по качеству:
1 место: П.2 (только численные с масштабированием)

- MSE: 876 млн, MAPE: 8.77%

- Наилучшее качество предсказаний

2 место: П.1 (только численные без масштабирования)

- MSE: 1246 млн, MAPE: 10.46%

- Худшее, но стабильное качество

3 место: П.4 (масштабирование + категориальные)

- MSE: 1547 млн, MAPE: 12.84%

- Лучше чем П.3, но хуже чем П.1

4 место: П.3 (категориальные без правильного масштабирования)

- MSE: 1617 млн, MAPE: 13.06%

- Наихудшее качество

2. Ключевые наблюдения:
Неожиданный результат: Добавление категориальных признаков даже с правильным масштабированием не улучшило, а ухудшило качество модели по сравнению с использованием только численных признаков.

Сравнение П.2 vs П.4:

- MSE ухудшилась на 76.4% (с 877 млн до 1547 млн)

- MAPE ухудшился на 46.5% (с 8.77% до 12.84%)

3. Объяснение результатов:
Почему категориальные признаки не помогают kNN в этой задаче:

__Проклятие размерности:__

После OneHotEncoding создается 100+ дополнительных признаков

В высокоразмерном пространстве расстояния становятся менее информативными

__Разбавление сигнала:__

Полезные численные признаки "разбавляются" множеством бинарных категориальных признаков

Модель теряет способность выделять значимые закономерности

__Специфика данных:__

Численные признаки (площадь, год постройки, количество комнат) могут быть более информативными для предсказания цены дома

Категориальные признаки добавляют шум

__Особенности алгоритма kNN:__

Чувствителен к соотношению информативных и шумовых признаков

Плохо справляется с разреженными высокоразмерными данными

### Выводы

1. Лучший подход: П.2 (только численные признаки с масштабированием)

Обоснование:

- Наилучшие результаты на валидационной выборке

- Стабильное качество across всех фолдов

- Простота реализации и интерпретации

2. Почему комбинированный подход не сработал:
- kNN неэффективен с высокоразмерными данными после OneHotEncoding

- Категориальные признаки не содержат достаточной дополнительной информации для компенсации роста размерности

- Численные признаки доминируют в предсказательной силе модели

### Заключение:

В данной задаче предсказания цен на дома с использованием алгоритма kNN оптимальным оказался подход использования только масштабированных численных признаков. Категориальные признаки, несмотря на правильную предобработку, ухудшают качество модели из-за проблем, связанных с высокой размерностью и особенностями работы алгоритма kNN.

# Пункт - 5 лучший дизайн + power transform (1 балл)

## Решение
В этом блоке необходимо написать код. Можно создать столько ячеек с кодом, сколько нужно для выполнения задания

In [ ]:
# Здесь код решения

## Ответ
В этом блоке необходимо вывести ответ для этой части задания. Ответ должен описаться на написанный код, построенные визуализации и проведенные расчеты. Если необходимо, то можно записать ответ в нескольких ячейках.

In [ ]:
# Здесь код подготовки ответа и сам ответ

# Пункт - 6 Все предыдущие приемы п1-п5 + power transform (target)

## Решение
В этом блоке необходимо написать код. Можно создать столько ячеек с кодом, сколько нужно для выполнения задания

In [ ]:
# Здесь код решения

## Ответ
В этом блоке необходимо вывести ответ для этой части задания. Ответ должен описаться на написанный код, построенные визуализации и проведенные расчеты. Если необходимо, то можно записать ответ в нескольких ячейках.

In [ ]:
# Здесь код подготовки ответа и сам ответ

# Пункт - 7 лучший дизайн + обработка выбросов (1 балл)

## Решение
В этом блоке необходимо написать код. Можно создать столько ячеек с кодом, сколько нужно для выполнения задания

In [ ]:
# Здесь код решения

## Ответ
В этом блоке необходимо вывести ответ для этой части задания. Ответ должен описаться на написанный код, построенные визуализации и проведенные расчеты. Если необходимо, то можно записать ответ в нескольких ячейках.

In [ ]:
# Здесь код подготовки ответа и сам ответ

# Пункт - 8 лучший дизайн + подбор гиперпараметров (1 балл)

## Решение
В этом блоке необходимо написать код. Можно создать столько ячеек с кодом, сколько нужно для выполнения задания

In [ ]:
# Здесь код решения

## Ответ
В этом блоке необходимо вывести ответ для этой части задания. Ответ должен описаться на написанный код, построенные визуализации и проведенные расчеты. Если необходимо, то можно записать ответ в нескольких ячейках.

In [ ]:
# Здесь код подготовки ответа и сам ответ